In [1]:
# import dask.dataframe as dd


In [2]:
!ls

drive  sample_data


In [3]:
# !unzip "/content/drive/MyDrive/Fake News Dataset/news_cleaned_2018_02_13.csv.zip"

In [4]:
#  df = dd.read_csv("/content/news_cleaned_2018_02_13.csv", on_bad_lines = "warn", encoding="latin1", engine='python',storage_options={"anon": True, 'use_ssl': True})

# split_data = df.repartition(npartitions=10)

# # Save each partition as a separate CSV file
# for i, partition in enumerate(split_data.to_delayed()):
#     partition.compute().to_csv(f'partition_{i}.csv', index=False)

In [ ]:
!pwd

/content


In [ ]:
! cp "/content/news_cleaned_2018_02_13.csv" "/content/drive/MyDrive/Fake News Dataset/."

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [34]:
df = spark.read.option("multiline",True).option("mode", "DROPMALFORMED").option("dateFormat", "yyyy-MM-dd").option("timestampFormat", "yyyy-MM-dd HH:mm:ss").csv("/content/drive/MyDrive/Fake News Dataset/news_cleaned_2018_02_13.csv", header=True)

# Split the DataFrame into smaller chunks based on a column (e.g., 'id')
# split_col = 'id'
# split_values = df.select(split_col).distinct().rdd.flatMap(lambda x: x).collect()

# # Split the DataFrame based on the column values and save as separate CSV files
# for value in split_values:
#     split_df = df.filter(df[split_col] == value)
#     split_df.write.csv(f"split_{value}.csv", header=True)

In [24]:
df.drop(11, axis=0)
# df = df.filter(("_2")<2 or numdf("_2")>2).show()

TypeError: ignored

In [15]:
df.createOrReplaceTempView("newssql")

In [20]:
query = spark.sql(""" 
SELECT * from newssql where id=137
""")

In [21]:
query.show()

KeyboardInterrupt: ignored

In [35]:
df.show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+----+-------+------+
|                 _c0|      id|              domain|                type|                 url|             content|          scraped_at|         inserted_at|          updated_at|               title|             authors|keywords|       meta_keywords|    meta_description|tags|summary|source|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+----+-------+------+
|                   0|       2|       express.co.uk|               rumor|https://www.expre...|Life is an illusi...|2018-01-2

In [36]:
 df_filtered = df.filter(col("_c0") != 10).filter(col("_c0") != 11)

In [37]:
df_filtered = df_filtered.filter(col("id").cast("int").isNotNull())

In [38]:
df_filtered.show(200)

+---+---+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+-------+------+
|_c0| id|              domain|      type|                 url|             content|          scraped_at|         inserted_at|          updated_at|               title|             authors|keywords|       meta_keywords|    meta_description|                tags|summary|source|
+---+---+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+-------+------+
|  0|  2|       express.co.uk|     rumor|https://www.expre...|Life is an illusi...|2018-01-25 16:17:...|2018-02-02 01:19:...|2018-02-02 01:19:...|Is life an ILLUSI...|     

In [7]:
# Scraped_at, inserted_at, updated_at, keywords, meta_keywords, meta_description
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count


In [15]:
null_percentage = df.agg(*[
    (1 - (count(col(c)).cast("double") / count("*"))).alias(c + '_null_percentage')
    for c in df.columns
])

# Show the null value percentages
null_percentage.show(truncate=False)



KeyboardInterrupt: ignored

In [ ]:
df.drop("scraped_at")

_c0,id,domain,type,url,content,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,2,express.co.uk,rumor,https://www.expre...,Life is an illusi...,null,null,null,null,null,null,null,null,null,null
They finally have...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
In 1978,Mr Wheeler’s pro...,unlike a wave – ...,null,null,null,null,null,null,null,null,null,null,null,null,null
At the time,the technology w...,but scientists h...,2018-01-25 16:17:...,2018-02-02 01:19:...,2018-02-02 01:19:...,Sean Martin,null,[''],THE UNIVERSE ceas...,null,null,null,null,null,null
1,6,barenakedislam.com,hate,http://barenakedi...,"Unfortunately, he...",2018-02-02 01:19:...,2018-02-02 01:19:...,Donald Trump,"Linda Rivera, Con...",null,[''],null,null,null,null
2,7,barenakedislam.com,hate,http://barenakedi...,The Los Angeles P...,2018-02-02 01:19:...,2018-02-02 01:19:...,Donald Trump,"Linda Rivera, Con...",null,[''],null,null,null,null
3,8,barenakedislam.com,hate,http://barenakedi...,The White House h...,null,null,null,null,null,null,null,null,null,null
Debka (h/t Marvin...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Last week,two Arab crown p...,Saudi Muhammed b...,summoned Abbas t...,Sheikh Tamim Al ...,who conferred wi...,null,null,null,null,null,null,null,null,null,null
The Israeli-Pales...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
df.drop("inserted_at")

_c0,id,domain,type,url,content,scraped_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,2,express.co.uk,rumor,https://www.expre...,Life is an illusi...,null,null,null,null,null,null,null,null,null,null
They finally have...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
In 1978,Mr Wheeler’s pro...,unlike a wave – ...,null,null,null,null,null,null,null,null,null,null,null,null,null
At the time,the technology w...,but scientists h...,2018-01-25 16:17:...,2018-02-02 01:19:...,2018-02-02 01:19:...,Is life an ILLUSI...,null,[''],THE UNIVERSE ceas...,null,null,null,null,null,null
1,6,barenakedislam.com,hate,http://barenakedi...,"Unfortunately, he...",2018-01-25 16:17:...,2018-02-02 01:19:...,Donald Trump,"Linda Rivera, Con...",null,[''],null,null,null,null
2,7,barenakedislam.com,hate,http://barenakedi...,The Los Angeles P...,2018-01-25 16:17:...,2018-02-02 01:19:...,Donald Trump,"Linda Rivera, Con...",null,[''],null,null,null,null
3,8,barenakedislam.com,hate,http://barenakedi...,The White House h...,null,null,null,null,null,null,null,null,null,null
Debka (h/t Marvin...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Last week,two Arab crown p...,Saudi Muhammed b...,summoned Abbas t...,Sheikh Tamim Al ...,who conferred wi...,first revealed h...,null,null,null,null,null,null,null,null,null
The Israeli-Pales...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
df.drop('updated_at')

_c0,id,domain,type,url,content,scraped_at,inserted_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,2,express.co.uk,rumor,https://www.expre...,Life is an illusi...,null,null,null,null,null,null,null,null,null,null
They finally have...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
In 1978,Mr Wheeler’s pro...,unlike a wave – ...,null,null,null,null,null,null,null,null,null,null,null,null,null
At the time,the technology w...,but scientists h...,2018-01-25 16:17:...,2018-02-02 01:19:...,2018-02-02 01:19:...,Is life an ILLUSI...,Sean Martin,[''],THE UNIVERSE ceas...,null,null,null,null,null,null
1,6,barenakedislam.com,hate,http://barenakedi...,"Unfortunately, he...",2018-01-25 16:17:...,2018-02-02 01:19:...,Donald Trump,"Linda Rivera, Con...",null,[''],null,null,null,null
2,7,barenakedislam.com,hate,http://barenakedi...,The Los Angeles P...,2018-01-25 16:17:...,2018-02-02 01:19:...,Donald Trump,"Linda Rivera, Con...",null,[''],null,null,null,null
3,8,barenakedislam.com,hate,http://barenakedi...,The White House h...,null,null,null,null,null,null,null,null,null,null
Debka (h/t Marvin...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
Last week,two Arab crown p...,Saudi Muhammed b...,summoned Abbas t...,Sheikh Tamim Al ...,who conferred wi...,first revealed h...,null,null,null,null,null,null,null,null,null
The Israeli-Pales...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [40]:
# df = df.drop("scraped_at", "updated_at", "inserted_at","keywords", "meta_keywords", "meta_description")
# df_filtered = df_filtered.drop("tags", "summary", "source")
df_filtered = df_filtered.drop("_c0", "keywords", "meta_keywords")

In [47]:
df_filtered.show()

+---+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|              domain|      type|                 url|             content|          scraped_at|         inserted_at|          updated_at|               title|             authors|    meta_description|
+---+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  2|       express.co.uk|     rumor|https://www.expre...|Life is an illusi...|2018-01-25 16:17:...|2018-02-02 01:19:...|2018-02-02 01:19:...|Is life an ILLUSI...|         Sean Martin|THE UNIVERSE ceas...|
|  6|  barenakedislam.com|      hate|http://barenakedi...|Unfortunately, he...|2018-01-25 16:17:...|2018-02-02 01:19:...|2018-02-02 01:19:...|        Donald Trump|Linda Rivera,

In [42]:
df_filtered = df_filtered.dropna(subset=["type"])



In [48]:
df_filtered.count()

KeyboardInterrupt: ignored

In [49]:
df_filtered.write.option("header",True) \
 .csv("/content/drive/MyDrive/Fake News Dataset/filtered_news_2.csv")

# df_filtered.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/Fake News Dataset/filtered_news_.parquet")


In [ ]:
!cd drive/Mydrive

/bin/bash: line 0: cd: drive/Mydrive: No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
